In [3]:
!pip install afinn
!pip install vaderSentiment
!pip install sklearn

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from afinn import Afinn
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from collections import Counter
import itertools

from nltk.corpus import stopwords
import string
from nltk import wordpunct_tokenize

from nltk.stem.lancaster import LancasterStemmer
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np

from wordcloud import WordCloud

from sklearn.preprocessing import MinMaxScaler
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#leggo il dataset con pandas
df = pd.read_csv('reviews_small_timed.csv')

In [0]:
#applichiamo Afinn alle review e aggiungiamo lo score in una nuova colonna
afinn = Afinn()
df['afinn'] = df['text'].apply(afinn.score)
df[['afinn', 'text']].head()
values = df.afinn.value_counts()

In [7]:
df[['score','afinn']].head(10)

,score,afinn
0,5,16.0
1,1,-2.0
2,4,3.0
3,2,3.0
4,5,9.0
5,4,7.0
6,5,6.0
7,5,9.0
8,5,3.0
9,5,8.0


In [8]:
pd.crosstab(df.score, df.afinn)

afinn,-49.0,-43.0,-40.0,-35.0,-33.0,-32.0,-28.0,-27.0,-26.0,-25.0,-24.0,-23.0,-22.0,-21.0,-20.0,-19.0,-18.0,-17.0,-16.0,-15.0,-14.0,-13.0,-12.0,-11.0,-10.0,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,-1.0,0.0,1.0,2.0,3.0,4.0,5.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,70.0,71.0,72.0,73.0,75.0,77.0,78.0,79.0,80.0,82.0,83.0,85.0,91.0,95.0,96.0,97.0,104.0,108.0,131.0,164.0
score,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,1,2,3,2,6,4,3,3,4,0,4,4,7,9,13,15,15,18,25,34,65,76,107,137,183,234,308,359,532,635,627,953,687,716,635,536,462,...,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0,0,0,0,1,0,2,0,0,1,1,0,2,2,1,4,6,7,10,17,18,36,48,87,120,152,244,242,474,344,466,418,381,347,...,0,0,0,0,3,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,0,0,0,1,6,8,11,17,30,38,59,87,129,220,254,481,391,556,626,507,607,...,2,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,2,0,1,4,5,1,5,3,10,10,21,28,48,65,145,136,255,509,526,728,908,921,1052,...,5,6,3,1,0,1,1,2,1,2,2,1,2,5,1,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,3,0,3,2,4,7,4,5,12,31,32,24,56,83,120,180,289,516,751,1671,1625,2618,4356,3539,4871,...,14,8,9,10,11,13,6,1,4,3,3,47,5,6,5,9,5,3,1,2,2,1,6,3,2,2,2,1,4,3,1,2,1,1,2,2,2,0,1,1


In [9]:
#vogliamo ora portare i risultati in 3 classi: -1 , 0, +1
confusion = pd.crosstab(np.sign(df.score -3), np.sign(df.afinn))
confusion

afinn,-1.0,0.0,1.0
score,,,
-1,4439,1427,9020
0,865,481,6713
1,2870,2180,72005


In [10]:
'''non più utilizzato ma potrebbe tornare utile
scaler = MinMaxScaler()
df['afinn normalized'] = scaler.fit_transform(df['afinn'].values.reshape(-1,1))
df[['score','afinn', 'afinn normalized']].head(10)
'''

"non più utilizzato ma potrebbe tornare utile\nscaler = MinMaxScaler()\ndf['afinn normalized'] = scaler.fit_transform(df['afinn'].values.reshape(-1,1))\ndf[['score','afinn', 'afinn normalized']].head(10)\n"

In [0]:
#applichiamo ora il vader score sentiment
analyzer = SentimentIntensityAnalyzer()
df['vader'] = df['text'].apply(analyzer.polarity_scores)


In [12]:
df[['score','afinn', 'vader']].head(10)

,score,afinn,vader
0,5,16.0,"{'neg': 0.0, 'neu': 0.695, 'pos': 0.305, 'comp..."
1,1,-2.0,"{'neg': 0.138, 'neu': 0.862, 'pos': 0.0, 'comp..."
2,4,3.0,"{'neg': 0.091, 'neu': 0.754, 'pos': 0.155, 'co..."
3,2,3.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,5,9.0,"{'neg': 0.0, 'neu': 0.552, 'pos': 0.448, 'comp..."
5,4,7.0,"{'neg': 0.029, 'neu': 0.809, 'pos': 0.163, 'co..."
6,5,6.0,"{'neg': 0.034, 'neu': 0.693, 'pos': 0.273, 'co..."
7,5,9.0,"{'neg': 0.0, 'neu': 0.52, 'pos': 0.48, 'compou..."
8,5,3.0,"{'neg': 0.0, 'neu': 0.851, 'pos': 0.149, 'comp..."
9,5,8.0,"{'neg': 0.0, 'neu': 0.705, 'pos': 0.295, 'comp..."


In [0]:
#accediamo come un dict all'oggetto compound che ci restituisce un valore tra -1 e 1
temp = []
for i in range(0,100000):
   temp.append(dict(df.iloc[i].vader)['compound'])
df['vader compound'] = temp

In [14]:
#Per andare a confrontare le due valutazioni diverse, andiamo a considerare le 3 classi: -1, 0, 1
#Per Afinn guarderemo il segno
#Per vader assegneremo lo 0 per valori compound t.c. -0.05 < compound < 0.05

temp = []
for i in range(0,100000):
  t = float(df.iloc[i]['vader compound'])
  if(t >= 0.05):
    temp.append('1')
  elif(t <= -0.05):
    temp.append('-1')
  else:
    temp.append('0')
df['vader polar'] = temp

df[['vader polar','vader compound']].head(20)

,vader polar,vader compound
0,1,0.9441
1,-1,-0.5664
2,1,0.8265
3,0,0.0000
4,1,0.9468
5,1,0.8830
6,1,0.9346
7,1,0.9487
8,1,0.6369
9,1,0.8313


In [15]:
temp = []
for i in range(0,100000):
  t = float(df.iloc[i]['afinn'])
  if(t > 0):
    temp.append('1')
  elif(t < 0):
    temp.append('-1')
  else:
    temp.append('0')
df['afinn polar'] = temp

df[['afinn polar','afinn']].head(20)

,afinn polar,afinn
0,1,16.0
1,-1,-2.0
2,1,3.0
3,1,3.0
4,1,9.0
5,1,7.0
6,1,6.0
7,1,9.0
8,1,3.0
9,1,8.0


In [16]:
df.drop(['vader','vader compound'], axis=1)

,productid,userid,profile,helpfulNum,helpfulDen,score,text,time,year,afinn,vader polar,afinn polar
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,I have bought several of the Vitality canned d...,2011-04-27,2011,16.0,1,1
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,Product arrived labeled as Jumbo Salted Peanut...,2012-09-07,2012,-2.0,-1,-1
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,This is a confection that has been around a fe...,2008-08-18,2008,3.0,1,1
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,If you are looking for the secret ingredient i...,2011-06-13,2011,3.0,0,1
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,Great taffy at a great price. There was a wid...,2012-10-21,2012,9.0,1,1
5,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,I got a wild hair for taffy and ordered this f...,2012-07-12,2012,7.0,1,1
6,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,This saltwater taffy had great flavors and was...,2012-06-20,2012,6.0,1,1
7,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,This taffy is so good. It is very soft and ch...,2012-05-03,2012,9.0,1,1
8,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,Right now I'm mostly just sprouting this so my...,2011-11-23,2011,3.0,1,1
9,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,This is a very healthy dog food. Good for thei...,2012-10-26,2012,8.0,1,1


In [17]:
#Andiamo quindi a plottare i confronti degli score e traiamo delle conclusioni
c1 = Counter(df['vader polar'])
c2 = Counter(df['afinn polar'])
print(c1)
print(c2)

'''come è possibile osservare dalle print, gli score positivi sono numericamente simili.
le due tecniche hanno trovato differenze un po più significative per gli score 
negativi e neutri.
'''

#plt.bar(c1.keys(), c1.values())
#plt.bar(c2.keys(), c2.values())

#verifichiamo dove gli score non combaciano
vader_positive = df[(df['vader polar']=='1')]
#vader_positive.head(10)
t = vader_positive[(vader_positive['afinn polar'] != '1')]
print(len(t)) #ben 4622 review sono classificate diversamente da afinn


Counter({'1': 87534, '-1': 10416, '0': 2050})
Counter({'1': 87738, '-1': 8174, '0': 4088})
4622


In [38]:
#volte in cui afinn sbaglia in negativo
subset_df = df[df['afinn polar'] == "-1"]
subsubset_df = subset_df[subset_df['score'] == 5]
print(subsubset_df.head(10))
print(len(subsubset_df))

      productid          userid  ... vader polar  afinn polar
11   B0009XLVG0  A2725IB4YY9JEB  ...          -1           -1
69   B000E7VI7S   AWCBF2ZWIN57F  ...          -1           -1
121  B003SE19UK  A3L15C30CBPZOR  ...          -1           -1
156  B0036VM05I   AUIVXYS8E4DEX  ...          -1           -1
185  B001KUUNP6  A18VJ0WOQAIZYD  ...          -1           -1
187  B00029XIZI  A2S72TUJDQUBMH  ...           1           -1
226  B007JFV6RK  A2WQBT7RS1HSSU  ...          -1           -1
292  B001UJEN6C  A3R6W1FEYDNT8V  ...           1           -1
295  B001UJEN6C   ADWJAJZTHNYN6  ...          -1           -1
328  B003VTN95K  A1J7ZSFU0SO3D4  ...          -1           -1

[10 rows x 14 columns]
2127


In [30]:
df.iloc[292].text

"I was looking for a natural energy shot that works, and I found it. Steaz energy shot is my favorite!<br /><br />I recently had gastric bypass surgery, so tons of sugar is a no-no. And I'm also allergic to Sucralose (Splenda) so most other energy drinks are out.<br /><br />I tried Bazi energy shots and they didn't work as well as Steaz, and definitely didn't taste as well. These have a taste that's almost like Coca-Cola--not bad at all.<br /><br />If you're looking for natural energy, give these a try."

In [39]:
#volte in cui vader sbaglia in negativo
subset_df = df[df['vader polar'] == "-1"]
subsubset_df = subset_df[subset_df['score'] == 5]
print(len(subsubset_df))

2244


In [40]:
subset_df = df[df['vader polar'] == "1"]
subsubset_df = subset_df[subset_df['score'] == 1]
print(subsubset_df.head(10))
print(len(subsubset_df))

      productid          userid  ... vader polar  afinn polar
12   B0009XLVG0   A327PCT23YH90  ...           1            0
73   B0059WXJKM  A25VFHVGI4CFTP  ...           1            1
75   B001EPPI84  A27TZ4WBU7N0YF  ...           1           -1
153  B002HQAXUW  A17ESTP5T7T1PH  ...           1            1
164  B000ITVLE2   ARTBQH9WOZTIV  ...           1            1
208  B0028C44Z0  A3BJ9NS09YGQT5  ...           1            1
229  B001L4ELRW  A1AK335P17JXL0  ...           1            1
233  B003ZFXJDW  A2KPPGFMPD20M6  ...           1            0
254  B0048IC328  A1W867A8DSHFHC  ...           1            0
282  B001D07IPG  A27UE9HJRZ22YI  ...           1            1

[10 rows x 14 columns]
4626


In [44]:
df.iloc[229].text

"This candy is not as described. The middle is almost hard it is not a silky or smooth filling as described.<br />Looks and tastes like it's way past it's expiration date.<br />Would never reccommend this. Paid a good chunk of cash for nothing"

In [41]:
subset_df = df[df['afinn polar'] == "1"]
subsubset_df = subset_df[subset_df['score'] == 1]
print(len(subsubset_df))

4928


In [63]:
#lunghezza delle review in media anno per anno
import statistics

df['len'] = df['text'].apply(len)

for i in range(2000,2013):
  different_year = df[df['year'] == i]
  sumL = 0
  for j in range(0,len(different_year)):
    sumL = sumL + different_year.iloc[j].len
  if(i != 2001 and i != 2002):
    print(sumL/len(different_year))

334.6666666666667
652.4117647058823
473.44444444444446
485.53478260869565
451.4231111111111
348.82560590094835
424.5794589340677
439.8252615265401
428.0940218525993
454.61941074523395
450.70147293902716


In [68]:
#lunghezza delle review in media anno per anno by score
for i in range(2000,2013):
  different_year = df[df['year'] == i]
  for score in range(1,6):
    different_score = different_year[different_year['score']==score]
    sumL = 0

    for j in range(0,len(different_score)):
      sumL = sumL + different_year.iloc[j].len
    if(i != 2001 and i != 2002):
      print("anno: ", i," score: ",score, " lunghezza media: ", sumL/len(different_year))

anno:  2000  score:  1  lunghezza media:  0.0
anno:  2000  score:  2  lunghezza media:  0.0
anno:  2000  score:  3  lunghezza media:  0.0
anno:  2000  score:  4  lunghezza media:  0.0
anno:  2000  score:  5  lunghezza media:  334.6666666666667
anno:  2003  score:  1  lunghezza media:  0.0
anno:  2003  score:  2  lunghezza media:  0.0
anno:  2003  score:  3  lunghezza media:  0.0
anno:  2003  score:  4  lunghezza media:  116.17647058823529
anno:  2003  score:  5  lunghezza media:  543.7058823529412
anno:  2004  score:  1  lunghezza media:  23.506172839506174
anno:  2004  score:  2  lunghezza media:  4.432098765432099
anno:  2004  score:  3  lunghezza media:  29.666666666666668
anno:  2004  score:  4  lunghezza media:  36.55555555555556
anno:  2004  score:  5  lunghezza media:  349.5308641975309
anno:  2005  score:  1  lunghezza media:  9.265217391304347
anno:  2005  score:  2  lunghezza media:  36.5
anno:  2005  score:  3  lunghezza media:  18.804347826086957
anno:  2005  score:  4  lun